In [7]:
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import requests 


In [8]:
def count_objects_in_polygon(polygon):
    if not isinstance(polygon, (Polygon, MultiPolygon)):
        raise TypeError("Not polygon obj")

    tags = {
        'aeroway': 'aerodrome',
        'seamark:type': 'harbour',
        'highway': 'motorway' #if we set this to True, we will get all roads, this is just highways
    }

    objects = ox.features_from_polygon(polygon, tags)

    num_airports = len(objects[objects['aeroway'] == 'aerodrome'])
    num_harbors = len(objects[objects['seamark:type'] == 'harbour'])
    num_highways = len(objects[objects['highway'].notna()])

    return {
        'airports': num_airports,
        'harbors': num_harbors,
        'highways': num_highways
    }


In [9]:

def bounding_coordinates(place, country): # this is from the vegetation notebook, the current one in the repo is corupt. once its not corupt we can import this func
    url = f"https://nominatim.openstreetmap.org/search.php?q={place}+{country}&polygon_geojson=1&format=json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        polygon_coordinates = 0
        bounding_coordinates = 0
        if(data[0]["class"] != "boundary"):
          for i in range(len(data)):
            if(data[i]["class"] == "boundary"):
              polygon_coordinates = data[i]['geojson']['coordinates'][0]
        else:
          polygon_coordinates = data[0]['geojson']['coordinates'][0]
        bounding_coordinates = [float(i) for i in data[0]['boundingbox']]
        return [polygon_coordinates, bounding_coordinates]

In [10]:
def flatten_coords(coord_list): #ChatGPT generated FYI, sorry ): , had to run to class
    """ Flatten the list of coordinates to a list of tuples. """
    # This function should handle nested structures appropriately
    if isinstance(coord_list[0], list):
        # If the first item is a list, we assume it's a list of lists
        if isinstance(coord_list[0][0], list):
            # If the nested list also contains a list, recurse deeper
            return flatten_coords([item for sublist in coord_list for item in sublist])
        else:
            # Otherwise, it's just one level deep of coordinate pairs
            return [tuple(item) for item in coord_list]
    else:
        # Base case: if we're at the coordinate pair level, stop flattening
        return [tuple(coord_list)]


In [11]:
poly_coords = bounding_coordinates('Rhode Island', 'USA')[0]
poly_coords = flatten_coords(poly_coords)
poly = Polygon(poly_coords)


object_counts = count_objects_in_polygon(poly)

print("Number of airports:", object_counts['airports'])
print("Number of harbors:", object_counts['harbors'])
print("Number of highways:", object_counts['highways'])

Number of airports: 12
Number of harbors: 40
Number of highways: 1023
